In [2]:
import time
import numpy as np
import pyaudio
import config


def start_stream(callback):
    p = pyaudio.PyAudio()
    frames_per_buffer = int(config.MIC_RATE / config.FPS)
    stream = p.open(format=pyaudio.paInt16,
                    channels=1,
                    rate=config.MIC_RATE,
                    input=True,
                    frames_per_buffer=frames_per_buffer)
    overflows = 0
    prev_ovf_time = time.time()
    while True:
        try:
            y = np.fromstring(stream.read(frames_per_buffer, exception_on_overflow=False), dtype=np.int16)
            y = y.astype(np.float32)
            stream.read(stream.get_read_available(), exception_on_overflow=False)
            callback(y)
        except IOError:
            overflows += 1
            if time.time() > prev_ovf_time + 1:
                prev_ovf_time = time.time()
                print('Audio buffer has overflowed {} times'.format(overflows))
    stream.stop_stream()
    stream.close()
    p.terminate()

In [3]:
"""This module implements a Mel Filter Bank.
In other words it is a filter bank with triangular shaped bands
arnged on the mel frequency scale.
An example ist shown in the following figure:
.. plot::
    from pylab import plt
    import melbank
    f1, f2 = 1000, 8000
    melmat, (melfreq, fftfreq) = melbank.compute_melmat(6, f1, f2, num_fft_bands=4097)
    fig, ax = plt.subplots(figsize=(8, 3))
    ax.plot(fftfreq, melmat.T)
    ax.grid(True)
    ax.set_ylabel('Weight')
    ax.set_xlabel('Frequency / Hz')
    ax.set_xlim((f1, f2))
    ax2 = ax.twiny()
    ax2.xaxis.set_ticks_position('top')
    ax2.set_xlim((f1, f2))
    ax2.xaxis.set_ticks(melbank.mel_to_hertz(melfreq))
    ax2.xaxis.set_ticklabels(['{:.0f}'.format(mf) for mf in melfreq])
    ax2.set_xlabel('Frequency / mel')
    plt.tight_layout()
    fig, ax = plt.subplots()
    ax.matshow(melmat)
    plt.axis('equal')
    plt.axis('tight')
    plt.title('Mel Matrix')
    plt.tight_layout()
Functions
---------
"""

from numpy import abs, append, arange, insert, linspace, log10, round, zeros


def hertz_to_mel(freq):
    """Returns mel-frequency from linear frequency input.
    Parameter
    ---------
    freq : scalar or ndarray
        Frequency value or array in Hz.
    Returns
    -------
    mel : scalar or ndarray
        Mel-frequency value or ndarray in Mel
    """
    return 2595.0 * log10(1 + (freq / 700.0))


def mel_to_hertz(mel):
    """Returns frequency from mel-frequency input.
    Parameter
    ---------
    mel : scalar or ndarray
        Mel-frequency value or ndarray in Mel
    Returns
    -------
    freq : scalar or ndarray
        Frequency value or array in Hz.
    """
    return 700.0 * (10**(mel / 2595.0)) - 700.0


def melfrequencies_mel_filterbank(num_bands, freq_min, freq_max, num_fft_bands):
    """Returns centerfrequencies and band edges for a mel filter bank
    Parameters
    ----------
    num_bands : int
        Number of mel bands.
    freq_min : scalar
        Minimum frequency for the first band.
    freq_max : scalar
        Maximum frequency for the last band.
    num_fft_bands : int
        Number of fft bands.
    Returns
    -------
    center_frequencies_mel : ndarray
    lower_edges_mel : ndarray
    upper_edges_mel : ndarray
    """

    mel_max = hertz_to_mel(freq_max)
    mel_min = hertz_to_mel(freq_min)
    delta_mel = abs(mel_max - mel_min) / (num_bands + 1.0)
    frequencies_mel = mel_min + delta_mel * arange(0, num_bands + 2)
    lower_edges_mel = frequencies_mel[:-2]
    upper_edges_mel = frequencies_mel[2:]
    center_frequencies_mel = frequencies_mel[1:-1]
    return center_frequencies_mel, lower_edges_mel, upper_edges_mel


def compute_melmat(num_mel_bands=12, freq_min=64, freq_max=8000,
                   num_fft_bands=513, sample_rate=16000):
    """Returns tranformation matrix for mel spectrum.
    Parameters
    ----------
    num_mel_bands : int
        Number of mel bands. Number of rows in melmat.
        Default: 24
    freq_min : scalar
        Minimum frequency for the first band.
        Default: 64
    freq_max : scalar
        Maximum frequency for the last band.
        Default: 8000
    num_fft_bands : int
        Number of fft-frequenc bands. This ist NFFT/2+1 !
        number of columns in melmat.
        Default: 513   (this means NFFT=1024)
    sample_rate : scalar
        Sample rate for the signals that will be used.
        Default: 44100
    Returns
    -------
    melmat : ndarray
        Transformation matrix for the mel spectrum.
        Use this with fft spectra of num_fft_bands_bands length
        and multiply the spectrum with the melmat
        this will tranform your fft-spectrum
        to a mel-spectrum.
    frequencies : tuple (ndarray <num_mel_bands>, ndarray <num_fft_bands>)
        Center frequencies of the mel bands, center frequencies of fft spectrum.
    """
    center_frequencies_mel, lower_edges_mel, upper_edges_mel =  \
        melfrequencies_mel_filterbank(
            num_mel_bands,
            freq_min,
            freq_max,
            num_fft_bands
        )

    center_frequencies_hz = mel_to_hertz(center_frequencies_mel)
    lower_edges_hz = mel_to_hertz(lower_edges_mel)
    upper_edges_hz = mel_to_hertz(upper_edges_mel)
    freqs = linspace(0.0, sample_rate / 2.0, num_fft_bands)
    melmat = zeros((num_mel_bands, num_fft_bands))

    for imelband, (center, lower, upper) in enumerate(zip(
            center_frequencies_hz, lower_edges_hz, upper_edges_hz)):

        left_slope = (freqs >= lower) == (freqs <= center)
        melmat[imelband, left_slope] = (
            (freqs[left_slope] - lower) / (center - lower)
        )

        right_slope = (freqs >= center) == (freqs <= upper)
        melmat[imelband, right_slope] = (
            (upper - freqs[right_slope]) / (upper - center)
        )

    return melmat, (center_frequencies_mel, freqs)



In [4]:
class ExpFilter:
    """Simple exponential smoothing filter"""
    def __init__(self, val=0.0, alpha_decay=0.5, alpha_rise=0.5):
        """Small rise / decay factors = more smoothing"""
        assert 0.0 < alpha_decay < 1.0, 'Invalid decay smoothing factor'
        assert 0.0 < alpha_rise < 1.0, 'Invalid rise smoothing factor'
        self.alpha_decay = alpha_decay
        self.alpha_rise = alpha_rise
        self.value = val

    def update(self, value):
        if isinstance(self.value, (list, np.ndarray, tuple)):
            alpha = value - self.value
            alpha[alpha > 0.0] = self.alpha_rise
            alpha[alpha <= 0.0] = self.alpha_decay
        else:
            alpha = self.alpha_rise if value > self.value else self.alpha_decay
        self.value = alpha * value + (1.0 - alpha) * self.value
        return self.value


def rfft(data, window=None):
    window = 1.0 if window is None else window(len(data))
    ys = np.abs(np.fft.rfft(data * window))
    xs = np.fft.rfftfreq(len(data), 1.0 / config.MIC_RATE)
    return xs, ys


def fft(data, window=None):
    window = 1.0 if window is None else window(len(data))
    ys = np.fft.fft(data * window)
    xs = np.fft.fftfreq(len(data), 1.0 / config.MIC_RATE)
    return xs, ys


def create_mel_bank():
    global samples, mel_y, mel_x
    samples = int(config.MIC_RATE * config.N_ROLLING_HISTORY / (2.0 * config.FPS))
    mel_y, (_, mel_x) = compute_melmat(num_mel_bands=config.N_FFT_BINS,
                                               freq_min=config.MIN_FREQUENCY,
                                               freq_max=config.MAX_FREQUENCY,
                                               num_fft_bands=samples,
                                               sample_rate=config.MIC_RATE)
    return mel_y, mel_x

In [5]:
MIC_RATE = 44100
FPS = 60
N_ROLLING_HISTORY = 2
N_FFT_BINS = 24
_time_prev = time.time() * 1000.0
_fps = ExpFilter(val=FPS, alpha_decay=0.2, alpha_rise=0.2)
prev_fps_update = time.time()

def frames_per_second():
    """Return the estimated frames per second

    Returns the current estimate for frames-per-second (FPS).
    FPS is estimated by measured the amount of time that has elapsed since
    this function was previously called. The FPS estimate is low-pass filtered
    to reduce noise.

    This function is intended to be called one time for every iteration of
    the program's main loop.

    Returns
    -------
    fps : float
        Estimated frames-per-second. This value is low-pass filtered
        to reduce noise.
    """
    global _time_prev, _fps
    time_now = time.time() * 1000.0
    dt = time_now - _time_prev
    _time_prev = time_now
    if dt == 0.0:
        return _fps.value
    return _fps.update(1000.0 / dt)

In [6]:

mel_smoothing = ExpFilter(np.tile(1e-1, N_FFT_BINS),
                         alpha_decay=0.5, alpha_rise=0.99)
from scipy.ndimage.filters import gaussian_filter1d


mel_gain = ExpFilter(np.tile(1e-1, N_FFT_BINS),
                         alpha_decay=0.01, alpha_rise=0.99)
fft_window = np.hamming(int(MIC_RATE / FPS) * N_ROLLING_HISTORY)

samples_per_frame = int(MIC_RATE / FPS)

y_roll = np.random.rand(config.N_ROLLING_HISTORY, samples_per_frame) / 1e16

MIN_VOLUME_THRESHOLD = 1e-10


def microphone_update(audio_samples):
    global y_roll, prev_rms, prev_exp, prev_fps_update
    # Normalize samples between 0 and 1
    y = audio_samples / 2.0**15
    # Construct a rolling window of audio samples
    y_roll[:-1] = y_roll[1:]
    y_roll[-1, :] = np.copy(y)
    y_data = np.concatenate(y_roll, axis=0).astype(np.float32)
    
    vol = np.max(np.abs(y_data))
    if vol < MIN_VOLUME_THRESHOLD:
        print('No audio input. Volume below threshold. Volume:', vol)
        #led.pixels = np.tile(0, (3, config.N_PIXELS))
        #led.update()
    else:
        # Transform audio input into the frequency domain
        N = len(y_data)
        N_zeros = 2**int(np.ceil(np.log2(N))) - N
        # Pad with zeros until the next power of two
        y_data *= fft_window
        y_padded = np.pad(y_data, (0, N_zeros), mode='constant')
        YS = np.abs(np.fft.rfft(y_padded)[:N // 2])
        # Construct a Mel filterbank from the FFT data
        mel_y, mel_x = create_mel_bank()
        mel = np.atleast_2d(YS).T * mel_y.T
        # Scale data to values more suitable for visualization
        # mel = np.sum(mel, axis=0)
        mel = np.sum(mel, axis=0)
        mel = mel**2.0
        # Gain normalization
        mel_gain.update(np.max(gaussian_filter1d(mel, sigma=1.0)))
        mel /= mel_gain.value
        mel = mel_smoothing.update(mel)
        # Map filterbank output onto LED strip
        #output = visualization_effect(mel)
        #led.pixels = output
        #led.update()
        
    
    
    if config.DISPLAY_FPS:
        fps = frames_per_second()
        if time.time() - 0.5 > prev_fps_update:
            prev_fps_update = time.time()
            print('FPS {:.0f} / {:.0f}'.format(fps, config.FPS))
    
# Call the start_stream function with the callback
start_stream(microphone_update)

/var/folders/bt/kswp83jx5mj4bn1bjkh9g4rsg_b7ks/T/ipykernel_4520/2111161650.py:3: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d
/var/folders/bt/kswp83jx5mj4bn1bjkh9g4rsg_b7ks/T/ipykernel_4520/997795979.py:19: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  y = np.fromstring(stream.read(frames_per_buffer, exception_on_overflow=False), dtype=np.int16)


No audio input. Volume below threshold. Volume: 9.987232e-17
FPS 48 / 60
No audio input. Volume below threshold. Volume: 0.0
No audio input. Volume below threshold. Volume: 0.0
No audio input. Volume below threshold. Volume: 0.0
No audio input. Volume below threshold. Volume: 0.0
No audio input. Volume below threshold. Volume: 0.0
No audio input. Volume below threshold. Volume: 0.0
No audio input. Volume below threshold. Volume: 0.0
No audio input. Volume below threshold. Volume: 0.0
No audio input. Volume below threshold. Volume: 0.0
No audio input. Volume below threshold. Volume: 0.0
No audio input. Volume below threshold. Volume: 0.0
No audio input. Volume below threshold. Volume: 0.0
No audio input. Volume below threshold. Volume: 0.0
No audio input. Volume below threshold. Volume: 0.0
No audio input. Volume below threshold. Volume: 0.0
No audio input. Volume below threshold. Volume: 0.0
No audio input. Volume below threshold. Volume: 0.0
No audio input. Volume below threshold. Vol

KeyboardInterrupt: 

In [2]:
import numpy as np
import pyqtgraph as pg
from PyQt5.QtWidgets import QApplication
from pyqtgraph.Qt import QtCore
from scipy.ndimage import gaussian_filter1d
import os
os.environ["QT_MAC_WANTS_LAYER"] = "1"
# Initialize the PyQt5 application
app = QApplication([])

# Create a window
win = pg.GraphicsLayoutWidget(show=True, title="Real-time Mel Filter Bank")
win.resize(800, 600)
win.setWindowTitle('Real-time Mel Filter Bank')

# Create a plot
plot = win.addPlot(title="Mel Filter Bank")
curve = plot.plot(pen='y')

# Define the callback function
def microphone_update(audio_data):
    global y_roll, fft_window, mel_gain, mel_smoothing, prev_fps_update
    
    # Construct a rolling window of audio samples
    y_roll[:-1] = y_roll[1:]
    y_roll[-1, :] = np.copy(audio_data)
    y_data = np.concatenate(y_roll, axis=0).astype(np.float32)
    
    vol = np.max(np.abs(y_data))
    print('Raw audio data:', audio_data)
    print('Volume:', vol)
    
    if vol < MIN_VOLUME_THRESHOLD:
        print('No audio input. Volume below threshold. Volume:', vol)
    else:
        # Transform audio input into the frequency domain
        N = len(y_data)
        N_zeros = 2**int(np.ceil(np.log2(N))) - N
        y_data *= fft_window
        y_padded = np.pad(y_data, (0, N_zeros), mode='constant')
        YS = np.abs(np.fft.rfft(y_padded)[:N // 2])
        mel = np.atleast_2d(YS).T * dsp.mel_y.T
        mel = np.sum(mel, axis=0)
        mel = mel**2.0
        mel_gain.update(np.max(gaussian_filter1d(mel, sigma=1.0)))
        mel /= mel_gain.value
        mel = mel_smoothing.update(mel)
        
        # Update the plot
        curve.setData(mel)
        plot.setYRange(0, np.max(mel) + 1)
    
    if config.DISPLAY_FPS:
        fps = frames_per_second()
        if time.time() - 0.5 > prev_fps_update:
            prev_fps_update = time.time()
            print('FPS {:.0f} / {:.0f}'.format(fps, config.FPS))

# Call the start_stream function with the callback
start_stream(microphone_update)


: 

In [1]:
import sys

from PyQt5.QtWidgets import QApplication, QWidget
app = QApplication(sys.argv)

window = QWidget()
window.show()
sys.exit(app.exec_())


SystemExit: 0

/Users/nalex2023/miniconda3/envs/vis/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import numpy as np
import pyqtgraph as pg
from PyQt5.QtWidgets import QApplication
from pyqtgraph.Qt import QtCore
import pyaudio

# Initialize the PyQt5 application
app = QApplication([])

# Create a window
win = pg.GraphicsLayoutWidget(show=True, title="Real-time Audio Plot")
win.resize(800, 600)
win.setWindowTitle('Real-time Audio Plot')

# Create a plot
plot = win.addPlot(title="Audio Signal")
curve = plot.plot(pen='y')

In [2]:
# PyAudio configuration
CHUNK = 1024  # Number of audio samples per frame
FORMAT = pyaudio.paInt16  # Audio format (16-bit PCM)
CHANNELS = 1  # Number of audio channels (mono)
RATE = 44100  # Sample rate (samples per second)

# Initialize PyAudio
p = pyaudio.PyAudio()

# Open audio stream
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

In [3]:
def update():
    # Read audio data from the stream
    data = np.frombuffer(stream.read(CHUNK), dtype=np.int16)
    # Update the plot with the new data
    curve.setData(data)
    plot.setYRange(-32768, 32767)  # Set y-axis range for 16-bit audio

# Create a timer to update the plot
timer = QtCore.QTimer()
timer.timeout.connect(update)
timer.start(50)  # Update every 50 ms

In [4]:
def update():
    # Read audio data from the stream
    data = np.frombuffer(stream.read(CHUNK), dtype=np.int16)
    # Update the plot with the new data
    curve.setData(data)
    plot.setYRange(-32768, 32767)  # Set y-axis range for 16-bit audio

# Create a timer to update the plot
timer = QtCore.QTimer()
timer.timeout.connect(update)
timer.start(50)  # Update every 50 ms

In [1]:
import numpy as np
import pyqtgraph as pg
from PyQt5.QtWidgets import QApplication
from pyqtgraph.Qt import QtCore
import pyaudio
import os
os.environ["QT_MAC_WANTS_LAYER"] = "1"

# Initialize the PyQt5 application
app = QApplication([])

# Create a window
win = pg.GraphicsLayoutWidget(show=True, title="Real-time Audio Plot")
win.resize(800, 600)
win.setWindowTitle('Real-time Audio Plot')

# Create a plot
plot = win.addPlot(title="Audio Signal")
curve = plot.plot(pen='y')

# PyAudio configuration
CHUNK = 1024  # Number of audio samples per frame
FORMAT = pyaudio.paInt16  # Audio format (16-bit PCM)
CHANNELS = 1  # Number of audio channels (mono)
RATE = 44100  # Sample rate (samples per second)

# Initialize PyAudio
p = pyaudio.PyAudio()

# Open audio stream
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK,
                input_device_index=0)

def update():
    # Read audio data from the stream
    data = np.frombuffer(stream.read(CHUNK), dtype=np.int16)
    # Update the plot with the new data
    curve.setData(data)
    plot.setYRange(-32768, 32767)  # Set y-axis range for 16-bit audio

# Create a timer to update the plot
timer = QtCore.QTimer()
timer.timeout.connect(update)
timer.start(50)  # Update every 50 ms

# Start the PyQt5 application
if __name__ == '__main__':
    app.exec_()

# Stop and close the audio stream
stream.stop_stream()
stream.close()
p.terminate()

OSError: [Errno -9981] Input overflowed

OSError: [Errno -9988] Stream closed

OSError: [Errno -9988] Stream closed

OSError: [Errno -9988] Stream closed

OSError: [Errno -9988] Stream closed

...

OSError: [Errno -9988] Stream closed

OSError: [Errno -9988] Stream closed

OSError: [Errno -9988] Stream closed

OSError: [Errno -9988] Stream closed

OSError: Stream not open